# <center><font color=slate>Properties and Class methods</font></center>

## <center><font color=tomato>Class</font> attributes vs <font color=tomato>instance</font> attributes</center>
Instance attributes are normally declared in the `__init__()` method
In the following example`nextSerial`is a class attribute. A class attribute is
called with the class name`ShippingContainer.nextSerial`, instance attributes would be`self.nextSerial` but it might produce a confusion in a more complex hierarchy of classes.

In [96]:
class ShippingContainer:
    next_serial = 1338

    def __init__(self, ownerCode, contents):
        self._ownerCode = ownerCode
        self._contents = contents
        self.serial = ShippingContainer.nextSerial
        ShippingContainer.nextSerial += 1

c1 = ShippingContainer(ownerCode='YML', contents='books')
c1._ownerCode, c1._contents, c1.serial

('YML', 'books', 1338)

In [97]:
c2 = ShippingContainer(ownerCode='MAE', contents='clothes')
c2._ownerCode, c2._contents, c2.serial

('MAE', 'clothes', 1339)

In [98]:
ShippingContainer.nextSerial

1340

## <center><font color=tomato>Static</font> methods with the`@staticmethod` decorator</center>

to create a function associated with the class, not the instance we create the static
method`_getNextSerial()`without the argument`self`.

In [99]:
class ShippingContainer:
    nextSerial = 1338

    @staticmethod
    def _getNextSerial():
        result = ShippingContainer.nextSerial
        ShippingContainer.nextSerial += 1
        return result

    def __init__(self, ownerCode, contents):
        self._ownerCode = ownerCode
        self._contents = contents
        self.serial = ShippingContainer._getNextSerial()

c1 = ShippingContainer(ownerCode='YML', contents='books')
c1._ownerCode, c1._contents, c1.serial

('YML', 'books', 1338)

In [100]:
c2 = ShippingContainer(ownerCode='MAE', contents='clothes')
c2._ownerCode, c2._contents, c2.serial

('MAE', 'clothes', 1339)

In [101]:
ShippingContainer.nextSerial


1340

## <center><font color=tomato>Class</font> methods with the`@classmethod` decorator</center>

As an alternative to`@staticmethod`we can use the `@classmethod`decorator,
which accept the class object as the first formal argument as follows:

In [102]:
class ShippingContainer:
    nextSerial = 1338

    @classmethod
    def _getNextSerial(cls):
        result = cls.nextSerial
        cls.nextSerial += 1
        return result

    def __init__(self, ownerCode, contents):
        self._ownerCode = ownerCode
        self._contents = contents
        self.serial = ShippingContainer._getNextSerial()

c1 = ShippingContainer(ownerCode='YML', contents='books')
c1._ownerCode, c1._contents, c1.serial

('YML', 'books', 1338)

In [103]:
c2 = ShippingContainer(ownerCode='MAE', contents='clothes')
c2._ownerCode, c2._contents, c2.serial

('MAE', 'clothes', 1339)

In [104]:
ShippingContainer.nextSerial

1340

### <center><font color=lightGreen>Choosing</font></center>

|`staticmethod`|`@classmethod`|
|-|-|
|No access needed to either class or instance objects|Requires access to the class object to call other class methods or the constructor|
|Most likely an implementation detail of the class||
|May be able to be moved to become a module-scope function||

If you need to refer to the class object within the method,
for example to access a class attribute prefer to use`@classmethod`.
If you do need to access class objects use`@staticmethod`

### Class methods for <font color=lightGreen>named constructors</font>
Also known as factory functions which construct objects with certain configurations.
In the following example we implement two named constructors: `createEmpty` and `createWithItems` methods

In [105]:
import iso6346

class ShippingContainer:
    nextSerial = 1338

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(ownerCode=ownerCode, serial=str(serial).zfill(6))

    @classmethod
    def _getNextSerial(cls):
        result = cls.nextSerial
        cls.nextSerial += 1
        return result

    @classmethod
    def createEmpty(cls, ownerCode):
        return cls(ownerCode=ownerCode, contents=None)

    @classmethod
    def createWithItems(cls, ownerCode, contents):
        return  cls(ownerCode=ownerCode, contents=contents)

    def __init__(self, ownerCode, contents):
        self._contents = contents
        self.bic = ShippingContainer._makeBicCode(
            ownerCode=ownerCode,
            serial=ShippingContainer._getNextSerial())

c1 = ShippingContainer.createEmpty(ownerCode='YML')
c1.bic

'YMLU0013380'

In [106]:
c2 = ShippingContainer.createWithItems(ownerCode='MAE', contents='clothes')
c2.bic

'MAEU0013392'

## <center>`@staticmethod`with <font color=tomato>Inheritance</font></center>
`@staticmethod` in python can be overwritten in subclasses
-   In the following example we have to change

`ShippingContainer._makeBicCode(ownerCode=ownerCode, serial=self._getNextSerial())`

to:

`self._makeBicCode(ownerCode=ownerCode, serial=self._getNextSerial())`

Because it is referring to a specific class, we need to get polymorphic overall behaviour.

In [107]:
import iso6346

class ShippingContainer:
    nextSerial = 1338

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(ownerCode=ownerCode, serial=str(serial).zfill(6))

    @classmethod
    def _getNextSerial(cls):
        result = cls.nextSerial
        cls.nextSerial += 1
        return result

    @classmethod
    def createEmpty(cls, ownerCode):
        return cls(ownerCode=ownerCode, contents=None)

    @classmethod
    def createWithItems(cls, ownerCode, contents):
        return  cls(ownerCode=ownerCode, contents=contents)

    def __init__(self, ownerCode, contents):
        self._contents = contents
        self.bic = self._makeBicCode(
            ownerCode=ownerCode,
            serial=self._getNextSerial())

class RefrigeratedShippingContainer(ShippingContainer):

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(
            ownerCode=ownerCode,
            serial=str(serial).zfill(6),
            category='R')

r1 = RefrigeratedShippingContainer.createEmpty(ownerCode='YML')
r1.bic


'YMLR0013388'

## <center>`@classmethod`with <font color=tomato>Inheritance</font></center>
`@classmethod`in python can be overwritten in subclasses

`@classmethod` behave polymorphic on subclasses,
no changed needed to be made, also because the`__init__`method
is inherited into the subclass.

In [108]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(
            ownerCode=ownerCode,
            serial=str(serial).zfill(6),
            category='R')

    @classmethod
    def createEmpty(cls, ownerCode, celsius):
        return cls(ownerCode=ownerCode, contents=None, celsius=celsius)

    @classmethod
    def createWithItems(cls, ownerCode, contents, celsius):
        return  cls(ownerCode=ownerCode, contents=contents, celsius=celsius)

    def __init__(self, ownerCode, contents, celsius):
        super().__init__(ownerCode=ownerCode, contents=contents)
        if celsius > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too high")
        self.celsius = celsius

r1 = RefrigeratedShippingContainer.createEmpty(ownerCode='YML', celsius=3)
r1.bic

'YMLR0013388'

In [109]:
r2 = RefrigeratedShippingContainer.createWithItems(ownerCode='YML', contents=['brocoli, orange'], celsius=3)
r1.bic

'YMLR0013388'

## <center><font color=tomato>encapsulation</font> using the`@property`decorator</center>
In the previous example celsius could be modify during running time

In [110]:
r2.celsius

3

In [111]:
r2.celsius = 12.0
r2.celsius

12.0

to correct this we can do the following:

In [112]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(
            ownerCode=ownerCode,
            serial=str(serial).zfill(6),
            category='R')

    @classmethod
    def createEmpty(cls, ownerCode, celsius):
        return cls(ownerCode=ownerCode, contents=None, celsius=celsius)

    @classmethod
    def createWithItems(cls, ownerCode, contents, celsius):
        return  cls(ownerCode=ownerCode, contents=contents, celsius=celsius)

    def __init__(self, ownerCode, contents, celsius):
        super().__init__(ownerCode=ownerCode, contents=contents)
        if celsius > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too high")
        self._celsius = celsius

    @property
    def celsius(self):
        return self._celsius

r1 = RefrigeratedShippingContainer.createWithItems(ownerCode='YML', contents=['brocoli, orange'], celsius=-18)
r1.celsius

-18

In [113]:
try:
    r1.celsius = 0
except AttributeError as e:
    print(e)

can't set attribute


### <font color=lightGreen>Set the attribute with </font>`setter`


In [114]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(
            ownerCode=ownerCode,
            serial=str(serial).zfill(6),
            category='R')

    @classmethod
    def createEmpty(cls, ownerCode, celsius):
        return cls(ownerCode=ownerCode, contents=None, celsius=celsius)

    @classmethod
    def createWithItems(cls, ownerCode, contents, celsius):
        return  cls(ownerCode=ownerCode, contents=contents, celsius=celsius)

    def __init__(self, ownerCode, contents, celsius):
        super().__init__(ownerCode=ownerCode, contents=contents)
        if celsius > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too high")
        self._celsius = celsius

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        if value > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too high")
        self._celsius = value

r1 = RefrigeratedShippingContainer.createWithItems(ownerCode='YML', contents=['brocoli, orange'], celsius=-18)
r1.celsius

-18

In [115]:
r1.celsius = -19
r1.celsius

-19

In [116]:
try:
    r1.celsius = 5
except ValueError as e:
    print(e)


Temperature too high


Moving forward, we will implement code to implement conversion between C and F

In [117]:
class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(
            ownerCode=ownerCode,
            serial=str(serial).zfill(6),
            category='R')

    @staticmethod
    def _cToF(celsius):
        return celsius * 9/5 + 32

    @staticmethod
    def _fToC(fahrenheit):
        return (fahrenheit - 32) * 5/9

    @classmethod
    def createEmpty(cls, ownerCode, celsius):
        return cls(ownerCode=ownerCode, contents=None, celsius=celsius)

    @classmethod
    def createWithItems(cls, ownerCode, items, celsius):
        return  cls(ownerCode=ownerCode, contents=list(items), celsius=celsius)

    def __init__(self, ownerCode, contents, celsius):
        super().__init__(ownerCode=ownerCode, contents=contents)
        self._celsius = celsius

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        if value > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too high")
        self._celsius = value

    @property
    def fahrenheit(self):
        return self._cToF(celsius=self.celsius)

    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = self._fToC(value)

r1 = RefrigeratedShippingContainer.createWithItems(ownerCode='YML', items=['brocoli, orange'], celsius=-18)
r1.celsius, r1.fahrenheit

(-18, -0.3999999999999986)

In [118]:
r1.fahrenheit = -10
r1.celsius, r1.fahrenheit

(-23.333333333333332, -10.0)

In [119]:
try:
    r1.celsius = 5
except ValueError as e:
    print(e)

Temperature too high


### <font color=lightGreen>Inheritance </font>interaction with the `@property` decorator
`@property` in python can be overwritten in subclasses
-   `getter` is straightforward
-   `setter` is much more involved

In [120]:
import iso6346

class ShippingContainer:

    HEIGHT_FT = 8.5
    WIDTH_FT = 8.0

    nextSerial = 1338

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(ownerCode=ownerCode, serial=str(serial).zfill(6))

    @classmethod
    def _getNextSerial(cls):
        result = cls.nextSerial
        cls.nextSerial += 1
        return result

    @classmethod
    def createEmpty(cls, ownerCode, lengthFt, *args, **kwargs):
        return cls(ownerCode=ownerCode, lengthFt=lengthFt, contents=None, *args, **kwargs)

    @classmethod
    def createWithItems(cls, ownerCode, lengthFt, items, *args, **kwargs):
        return  cls(ownerCode=ownerCode, lengthFt=lengthFt, contents=list(items), *args, **kwargs)

    def __init__(self, ownerCode, lengthFt, contents):
        self._contents = contents
        self._length = lengthFt
        self.bic = self._makeBicCode(
            ownerCode=ownerCode,
            serial=ShippingContainer._getNextSerial())

    @property
    def volumeFt3(self):
        return ShippingContainer.HEIGHT_FT\
               * ShippingContainer.WIDTH_FT\
               * self._length

c = ShippingContainer.createEmpty(ownerCode='YML', lengthFt=20)
c.volumeFt3


1360.0

In [121]:

class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0

    FRIDGE_VOLUME_FT3 = 100

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(
            ownerCode=ownerCode,
            serial=str(serial).zfill(6),
            category='R')

    @staticmethod
    def _cToF(celsius):
        return celsius * 9/5 + 32

    @staticmethod
    def _fToC(fahrenheit):
        return (fahrenheit - 32) * 5/9

    def __init__(self, ownerCode, lengthFt, contents, celsius):
        super().__init__(ownerCode=ownerCode, lengthFt=lengthFt, contents=contents)
        self.celsius = celsius

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        if value > RefrigeratedShippingContainer.MAX_CELSIUS:
            raise ValueError("Temperature too high")
        self._celsius = value

    @property
    def fahrenheit(self):
        return RefrigeratedShippingContainer._cToF(celsius=self.celsius)

    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = RefrigeratedShippingContainer._fToC(value)

    @property
    def volumeFt3(self):
        return super().volumeFt3\
               - RefrigeratedShippingContainer.FRIDGE_VOLUME_FT3


In [122]:
r = RefrigeratedShippingContainer.createEmpty(ownerCode='YML', lengthFt=20, celsius=0.0)
r.celsius


0.0

`@property setter` needs more work to be overwritten


In [123]:
class HeatedRefrigeratedShippingContainer(RefrigeratedShippingContainer):

    MIN_CELSIUS = -20

    @RefrigeratedShippingContainer.celsius.setter
    def celsius(self, value):
        if value < HeatedRefrigeratedShippingContainer.MIN_CELSIUS:
            raise ValueError('Temperature too cold')
        RefrigeratedShippingContainer.celsius.fset(self, value)

h = HeatedRefrigeratedShippingContainer.createEmpty(ownerCode='YML', lengthFt=40, celsius=-10.0)
h.celsius

-10.0

In [124]:
try:
    h.celsius = -21
except ValueError as e:
    print(e)

Temperature too cold


<h3><font color=lightGreen>Properties and the template method pattern</font></h3>

To avoid the calls to a top level class in `HeatedRefrigeratedShippingContainer`
we will refactor implementing the`_setCelsius()`function as follows:


In [125]:
import iso6346

class ShippingContainer:

    HEIGHT_FT = 8.5
    WIDTH_FT = 8.0

    nextSerial = 1338

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(ownerCode=ownerCode, serial=str(serial).zfill(6))

    @classmethod
    def _getNextSerial(cls):
        result = cls.nextSerial
        cls.nextSerial += 1
        return result

    @classmethod
    def createEmpty(cls, ownerCode, lengthFt, *args, **kwargs):
        return cls(ownerCode=ownerCode, lengthFt=lengthFt, contents=None, *args, **kwargs)

    @classmethod
    def createWithItems(cls, ownerCode, lengthFt, items, *args, **kwargs):
        return  cls(ownerCode=ownerCode, lengthFt=lengthFt, contents=list(items), *args, **kwargs)

    def __init__(self, ownerCode, lengthFt, contents):
        self._contents = contents
        self._length = lengthFt
        self.bic = self._makeBicCode(
            ownerCode=ownerCode,
            serial=ShippingContainer._getNextSerial())

    @property
    def volumeFt3(self):
        return ShippingContainer.HEIGHT_FT\
               * ShippingContainer.WIDTH_FT\
               * self._length


class RefrigeratedShippingContainer(ShippingContainer):

    MAX_CELSIUS = 4.0

    FRIDGE_VOLUME_FT3 = 100

    @staticmethod
    def _makeBicCode(ownerCode, serial):
        return iso6346.create(
            ownerCode=ownerCode,
            serial=str(serial).zfill(6),
            category='R')

    @staticmethod
    def _cToF(celsius):
        return celsius * 9/5 + 32

    @staticmethod
    def _fToC(fahrenheit):
        return (fahrenheit - 32) * 5/9

    def __init__(self, ownerCode, lengthFt, contents, celsius):
        super().__init__(ownerCode=ownerCode, lengthFt=lengthFt, contents=contents)
        self.celsius = celsius

    def _setCelsius(self, value):
        if value > RefrigeratedShippingContainer.MAX_CELSIUS:
                raise ValueError("Temperature too high")
        self._celsius = value

    @property
    def celsius(self):
        return self._celsius

    @celsius.setter
    def celsius(self, value):
        self._setCelsius(value)


    @property
    def fahrenheit(self):
        return RefrigeratedShippingContainer._cToF(celsius=self.celsius)

    @fahrenheit.setter
    def fahrenheit(self, value):
        self.celsius = RefrigeratedShippingContainer._fToC(value)

    @property
    def volumeFt3(self):
        return super().volumeFt3\
               - RefrigeratedShippingContainer.FRIDGE_VOLUME_FT3

class HeatedRefrigeratedShippingContainer(RefrigeratedShippingContainer):

    MIN_CELSIUS = -20

    def _setCelsius(self, value):
        if value < HeatedRefrigeratedShippingContainer.MIN_CELSIUS:
            raise ValueError('Temperature too cold')
        super()._setCelsius(value=value)

h = HeatedRefrigeratedShippingContainer.createEmpty(ownerCode='YML', lengthFt=40, celsius=-10.0)
h.celsius

-10.0

In [126]:
try:
    h.celsius = -21
except ValueError as e:
    print(e)

Temperature too cold


In [127]:
try:
    h.celsius = 14
except ValueError as e:
    print(e)

Temperature too high
